### Installing Requried Libaries

In [59]:
!pip install langchain_core
!pip install langchain_community
!pip install pypdf
!pip install chromadb
!pip install faiss-cpu
!pip install langchain
!pip install sentence_transformers
!pip install google-generativeai
!pip install langchain_google_genai

In [60]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

### Loading the model used gemini-1.5-pro-latest

In [61]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",google_api_key=GOOGLE_API_KEY,temperature=0.4)

### Loading the documents and converted into chunks

In [62]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("/content/drive/MyDrive/HOUND")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='The Hound of the Baskervilles \x18Chapter 1  \n \nMr. Sherlock Holmes \nMr. Sherlock Holmes, who was usually very late in the \nmornings, save upon those not infrequent occasions \nwhen he was up all night, was seated at the breakfast table. \nI stood upon the hearth-rug and picked up the stick which \nour visitor had left behind him the night before. It was a \nfine, thick piece of wood, bulbous-headed, of the sort which \nis known as a ‘Penang lawyer.’ Just under the head was a \nbroad silver band nearly an inch across. ‘To James Mortim -\ner, M.R.C.S., from his friends of the C.C.H.,’ was engraved \nupon it, with the date ‘1884.’ It was just such a stick as the \nold-fashioned family practitioner used to carry—dignified, \nsolid, and reassuring.\n‘Well, Watson, what do you make of it?’\nHolmes was sitting with his back to me, and I had given \nhim no sign of my occupation.\n‘How did you know what I was doing? I believe you have \neyes in the back of your head.

### Loading the GoogleGenerativeAIEmbeddings

In [63]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [64]:
embedding_model= GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY, model="models/embedding-001")


### VectorStore Creation

In [65]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents,embedding_model)

## query using similarity search

In [66]:
##query using similarity search

query = "who is the wife of Stapleton.what is her name"
relevant_doc = vectorstore.similarity_search(query)
relevant_doc[0].page_content

'1\x18\x18 Free eBooks at Planet eBook.comno one more easy to trace than a schoolmaster. There are \nscholastic agencies by which one may identify any man who \nhas been in the profession. A little investigation showed me \nthat a school had come to grief under atrocious circum -\nstances, and that the man who had owned it—the name was \ndifferent—had disappeared with his wife. The descriptions \nagreed. When I learned that the missing man was devoted \nto entomology the identification was complete.’\nThe darkness was rising, but much was still hidden by \nthe shadows.\n‘If this woman is in truth his wife, where does Mrs. Laura \nLyons come in?’ I asked.\n‘That is one of the points upon which your own research -\nes have shed a light. Your interview with the lady has cleared \nthe situation very much. I did not know about a projected \ndivorce between herself and her husband. In that case, re -\ngarding Stapleton as an unmarried man, she counted no \ndoubt upon becoming his wife.’'

In [67]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":5})

## creating the prompt template

In [68]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context.if the answer the dont know please give it has i dont the answer (naku telvadhu gurthu ledhu marchipoyina)

{context}
Question:{question}

Helpful Answers:
 """

In [69]:
prompt=PromptTemplate.from_template(prompt_template)

### creating the chain with RetriveaLIQA

In [70]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

## Results

In [79]:
query = "who is the wife of Stapleton."

In [80]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


## Who is Stapleton's Wife?

Based on the provided text, the woman who has been posing as **Miss Stapleton** is actually **Stapleton's wife**. Sherlock Holmes reveals this information to Watson, explaining that she has been under Stapleton's influence, possibly through a combination of love and fear. 



In [81]:
result['result']

"## Who is Stapleton's Wife?\n\nBased on the provided text, the woman who has been posing as **Miss Stapleton** is actually **Stapleton's wife**. Sherlock Holmes reveals this information to Watson, explaining that she has been under Stapleton's influence, possibly through a combination of love and fear. \n"

In [84]:
query = "tell me about watson."

In [85]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


## Watson: A Glimpse into Sherlock's Companion

Based on the provided text, here's what we can gather about Watson:

* **Close friend and confidante of Sherlock Holmes:**  The text refers to Watson as "my friend" and shows them spending time together, both professionally and casually. Watson is clearly someone Holmes trusts and relies on.
* **Observant and analytical:**  Although not possessing Holmes' level of deduction, Watson demonstrates an ability to observe and analyze. He notices details like the state of Holmes' chin and linen, indicating a keen eye.
* **Respectful of Holmes' abilities:**  Watson acknowledges and admires Holmes' exceptional skills, even referring to him as a "connoisseur" in one instance. He doesn't seem to be envious but rather appreciative of his friend's unique talents.
* **Actively involved in cases:**  Watson doesn't just passively observe; he participates in investigations and contributes his own insights. He even suggests visiting Holmes to discuss a "li

In [73]:
query = "where stapleton killed charles baskerville"

In [74]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


## Analyzing the Text for Location of Charles Baskerville's Death:

The provided text excerpts don't explicitly state the exact location where Stapleton killed Charles Baskerville. However, we can gather some clues and make inferences based on the information provided:

* **Sir Charles's fear of the moor:** The text mentions Sir Charles being superstitious and fearing the legend of the hound. This suggests he might avoid the moor, especially at night.
* **Stapleton's knowledge of the moor:** Stapleton, living near the moor and familiar with its paths, would have an advantage in choosing a location for the deed.
* **The Hound as the murder weapon:** The use of the hound implies a location suitable for its movement and concealment, likely somewhere on the moor.

**Inference:** Considering these points, it's highly probable that Stapleton killed Charles Baskerville somewhere on the moor, taking advantage of the environment and Sir Charles's fear. 

**However, the text doesn't provide a sp

In [75]:
query = "briefly explain me about sherlock homes"

In [76]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


## Sherlock Holmes: A Master Detective

Based on the provided text, here's what we can gather about Sherlock Holmes:

* **Brilliant Deductive Mind:** He excels at solving puzzles and mysteries that others find baffling. People seek him out for his unique ability to think critically and make connections others miss. 
* **Keen Observer:** He notices small details and uses them to piece together the bigger picture. He can deduce information about people and situations from seemingly insignificant clues. 
* **Skilled Investigator:** He actively investigates cases, going beyond just thinking to gather evidence and confront suspects. He utilizes various methods like observation, disguise, and even ambush to uncover the truth.
* **Logical and Rational:** He prioritizes logic and reason over emotions when solving cases. He carefully analyzes facts and evidence to arrive at well-founded conclusions.
* **Confident and Assured:** He trusts his abilities and deductions, often displaying a calm and

In [77]:
query = "briefly explain me about sai krishna"

In [78]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

## Sai Krishna in the Context:

Based on the provided excerpt, there is no mention of anyone named "Sai Krishna." Therefore, I am unable to provide information about this individual based on the given context. 

**naku telvadhu gurthu ledhu marchipoyina** 

